In [1]:
from simpleai.search import SearchProblem, simulated_annealing
import numpy as np

from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Cascadia Code; # Any monospaced font should work
        line-height: 1.3 !important;
        font-size: 14px !important;
    }
</style>
""")


In [20]:
SCALE = 10.0174
crater_map = np.load('crater_map.npy')
NR, NC = crater_map.shape

# Start positions
x0, y0 = 3350, 5800
MAX_DIFF = 0.5
UNTRAVERSABLE = -1
STEP_SIZE = 1


In [19]:
def pos_to_index(pos: list[int]) -> list[int]:
    nr = NR - round(pos[0] / SCALE)
    nc = round(pos[1] / SCALE)
    return [nr, nc]


def coord_to_state(coord: list[int]) -> str:
    return f'{coord[0]},{coord[1]}'


def state_to_coord(state: str) -> list[int]:
    return tuple([int(x) for x in state.split(',')])


def report(result):
    print(f'Found path with length {len(result.path())}')
    print(f'Path: {result.path()}')
    print(f'Cost: {result.cost}')


# def tester(search_method, imax, delta=20):

#     print(f'Testing {search_method.__name__}')
#     for i in np.linspace(imax, 0, delta):
#         problem = mars_route_finding(START_STATE, GOAL_STATE, mars_map, i * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)
#         result = search_method(problem, graph_search=True)
#         report(result)
#         print(f'Max difference (factor {i}): {i * MAX_DIFF}')


def distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)


In [7]:
class mars_route_finding(SearchProblem):

    def __init__(self, initial_state: str, mars_map: np.array, max_height: float, not_traversable: int, step_size: int):

        self.mars_map = mars_map
        self.initial_state = initial_state
        # Seems very convoluted, but this line of code finds the index associated with the smallest value on the array (not including zero)
        self.goal_state = coord_to_state(np.unravel_index(np.argmin(np.sqrt(crater_map * crater_map), axis=None), crater_map.shape))
        self.max_height = max_height
        self.not_traversable = not_traversable
        self.step_size = step_size

        print("Initial state: ", self.initial_state)
        print("Goal state: ", self.goal_state)

        SearchProblem.__init__(self, initial_state)

    def actions(self, state):
        mars_map = self.mars_map
        actions = []

        pos_directions = ['U', 'D', 'L', 'R', 'UL', 'UR', 'DL', 'DR']
        x0, y0 = state_to_coord(state)

        for action in pos_directions:
            x, y = state_to_coord(self.result(state, action))
            height_difference = abs(mars_map[x0, y0] - mars_map[x, y])

            if (mars_map[x, y] != self.not_traversable):
                if height_difference <= self.max_height:
                    # print(f'[{x}, {y}], action: {action}, height_difference: {height_difference}')
                    actions.append(action)
    
        return actions

    def result(self, state, action):
        x, y = state_to_coord(state)
        step_size = self.step_size

        if action.count('U'):
            x -= step_size
        elif action.count('D'):
            x += step_size
        elif action.count('L'):
            y -= step_size
        elif action.count('R'):
            y += step_size
        elif action.count('UL'):
            x -= step_size
            y -= step_size
        elif action.count('UR'):
            x -= step_size
            y += step_size
        elif action.count('DL'):
            x += step_size
            y -= step_size
        elif action.count('DR'):
            x += step_size
            y += step_size
        else:
            print('No se encontró la acción')
        
        new_state = coord_to_state([x, y])
        return new_state

    def is_goal(self, state):

        return state == self.goal_state

    def cost(self, state, action, state2):
        alpha, beta = 1, 1

        x0, y0 = state_to_coord(state)
        xn, yn = state_to_coord(state2)

        distance = np.sqrt((x0 - xn)**2 + (y0 - yn)**2)
        height_difference = abs(self.mars_map[x0, y0] - self.mars_map[xn, yn])

        return alpha * distance + beta * height_difference

    def heuristic(self, state):
        x, y = state_to_coord(state)
        xg, yg = state_to_coord(self.goal_state)
        distance = np.sqrt((x - xg)**2 + (y - yg)**2)

        return distance

In [8]:
crater_map.argmin()

0

In [16]:
index = np.unravel_index(np.argmin(np.sqrt(crater_map * crater_map), axis=None), crater_map.shape)

In [17]:
index

(58, 66)

In [18]:
crater_map[index]

0.09667470703125218

In [21]:
coord_to_state(index)

'58,66'